In [1]:
# Importing relevant modules
import psycopg2
import pandas as pd

In [12]:
# Function to create DB
def create_database():
    # Connect to default postgres DB
    conn = psycopg2.connect('host=127.0.0.1 dbname=postgres user=postgres password=1')
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # Create myFutbaallDB with UTF8 encoding
    cur.execute('DROP DATABASE IF EXISTS myFutbaallDB')
    cur.execute('CREATE DATABASE football')
    
    # Sever connection to default DB
    conn.close()
    
    # Connect to myFutbaallDB DB
    conn = psycopg2.connect('host=127.0.0.1 dbname=football user=postgres password=1')
    cur = conn.cursor()
    
    # Return cursor and connection
    return cur, conn

In [26]:
# Reading csv files in
apps = pd.read_csv('appearances.csv')
player_val = pd.read_csv('player_valuations.csv')

In [33]:
# Selecting relevant cols from apps df
apps = apps[['player_id', 'player_name', 'player_current_club_id', 'goals']]

In [34]:
# Selecting relevant cols from player_val df
player_val = player_val[['player_id', 'market_value_in_eur', 
                       'current_club_id']]

In [37]:
# Establishing a database connection and creating a cursor object
cur, conn = create_database()

In [87]:
# Create apps table
apps_create = ("CREATE TABLE IF NOT EXISTS apps(player_id varchar primary key, player_name varchar, player_current_club_id numeric, goals numeric)")

In [89]:
# Execute the SQL statement defined in 'apps_create' to create the 'apps' table
cur.execute(apps_create)
# Commit changes to DB
conn.commit()

In [51]:
# Prepare SQL insert into apps table
apps_insert = ("INSERT INTO apps( \
              player_id, player_name, player_current_club_id, goals \
              ) VALUES (%s, %s, %s, %s)")

In [53]:
# Insert rows from 'apps' DataFrame using prepared SQL statement
for i, rows in apps.iterrows():
    cur.execute(apps_insert, list(rows))

In [54]:
# Commit changes to DB
conn.commit()

In [45]:
# Create player_val table
player_val_create = ("CREATE TABLE IF NOT EXISTS player_val( \
                     player_id numeric, market_value_in_eur numeric, current_club_id numeric)")

In [46]:
# Execute the SQL statement defined in 'player_val_create' to create the 'player_val' table 
cur.execute(player_val_create)
# Commit changes to DB
conn.commit()

In [71]:
# Prepare SQL insert into player_val table
player_val_insert = ("INSERT INTO player_val( \
player_id, market_value_in_eur, current_club_id \
                    ) VALUES (%s, %s, %s)")

In [73]:
# Insert rows from 'player_val' DataFrame using prepared SQL statement
for i, rows in player_val.iterrows():
    cur.execute(player_val_insert, list(rows))

In [76]:
# Commit changes to DB
conn.commit()

In [90]:
# Close connection
conn.close()